## Case for shewanella
> 2023-01-29
### 1.import packages

In [1]:
import sys,os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
sys.path.append("../tools/")
import tools.funclib as funclib
import numpy as np
import pandas as pd
from keras.models import load_model
from tools.Attention import Attention

import config as cfg
import tools.embedding_esm as ebdesm
from pandarallel import pandarallel 
pandarallel.initialize() 

%load_ext autoreload
%autoreload 2

/home/shizhenkun/miniconda3/envs/DMLF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 52 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Read shewanella fasta file

In [2]:
data_shewanella = funclib.load_fasta_to_table(cfg.FILE_CASE_SHEWANELLA_FASTA)
data_shewanella.head(3)

,id,seq
0,WP_005054087.1,MTQPFNFEQALKDLQSGKSLTGKDSILGPLIKQLTEAALQAELEQH...
1,WP_006079870.1,MSRVCQVTGKKPMVGNNRSHAKNATRRRFLPNLQNHRFWLEEEKRF...
2,WP_006079871.1,MAKAKGNREKIKLVSTAKTGHFYTTEKNKRNMPEKMEIKKFDPVIR...


## 3. Embedding

In [3]:
# final_rep0, final_rep32, final_rep33 = ebdesm.get_rep_multi_sequence(data_shewanella)
# final_rep32.to_feather(cfg.RESULTSDIR+'ebdesm_32_shewanella.feather')


## 4.读取测试数据

In [4]:
# embeddin 方法
EMBD_METHO='esm32'
EMBD_WH = 1280
TOPK=20

ecs_2 = [True, False]
ecs_8 = [1,2,3,4,5,6,7,8]
label_model_ec = pd.read_feather(f'{cfg.MODELDIR}/task3_labels.feather').label_multi.to_list()

ebdesm_32_shewanella = pd.read_feather(cfg.RESULTSDIR+'ebdesm_32_shewanella.feather')
test_feature = ebdesm_32_shewanella.iloc[:, 1:]
test_feature =np.reshape(np.array(test_feature),(test_feature.shape[0],1,EMBD_WH))
test_res = pd.DataFrame(ebdesm_32_shewanella.id.copy().astype('str'))



## 5. DMLF [task1, task2, task3]

In [5]:
# task1
model_t1 = load_model(f'{cfg.MODELDIR}/task1_2022.h5', custom_objects={"Attention": Attention},compile=False)
predicted = model_t1.predict(test_feature)
test_res['dmlf_isenzyme'] = np.argmax(predicted, axis=-1).astype('bool')

# Task2
list_func = list()
model_t2 = load_model(f'{cfg.MODELDIR}/task2_model_2022_label8.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model_t2.predict(test_feature)
for i in range(predicted.shape[0]):
    list_func.append(sorted(dict(zip(set(ecs_8),predicted[i])).items(),key = lambda x:x[1], reverse = True)[0][0])
test_res['dmlf_functions']=list_func


#Task3
model_t3 = load_model(f'{cfg.MODELDIR}/task3_esm32_2022.h5',custom_objects={"Attention": Attention},compile=False)
predicted = model_t3.predict(test_feature)
test_res['sorted_ec'] = pd.DataFrame(predicted).parallel_apply(lambda x: sorted(dict(zip((label_model_ec), x.to_list())).items(),key = lambda x:x[1], reverse = True), axis=1)
test_res['ec_dmlf']=test_res.parallel_apply(lambda x: (list(dict(x.sorted_ec[0:x.dmlf_functions]).keys())), axis=1)
test_res['ec_dmlf']=test_res.parallel_apply(lambda x: '-' if x.dmlf_isenzyme ==False else ','.join(x.ec_dmlf), axis=1)
test_res['ec_dmlf_topk']=test_res.parallel_apply(lambda x: x.sorted_ec[0:TOPK], axis=1)

test_res

132/132 [==============================] - 1s 1ms/step


,id,dmlf_isenzyme,dmlf_functions,sorted_ec,ec_dmlf,ec_dmlf_topk
0,WP_005054087.1,False,1,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0..."
1,WP_006079870.1,False,1,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0..."
2,WP_006079871.1,False,1,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0..."
3,WP_006080725.1,False,1,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0...."
4,WP_006080791.1,False,1,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0...."
...,...,...,...,...,...,...
4197,WP_238560620.1,False,1,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0..."
4198,WP_238560621.1,False,1,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0..."
4199,WP_238560622.1,False,1,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00..."
4200,WP_238560623.1,False,1,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0..."


## 6.Balst

In [6]:
sprot = pd.read_feather(cfg.DATADIR+'uniprot/snap201802.feather')
data_shewanella = funclib.load_fasta_to_table(cfg.FILE_CASE_SHEWANELLA_FASTA)

res_diamond = funclib.getblast(train=sprot[['id','seq']], test=data_shewanella[['id','seq']])
res_diamond = res_diamond [['id','sseqid','pident']].merge(sprot[['id','ec_number']], how='left', left_on='sseqid', right_on='id')
res_diamond = res_diamond[['id_x', 'ec_number','pident']].rename(columns={'id_x':'id','ec_number':'ec_blast', 'pident':'pident_blast'})

test_res = test_res.merge(res_diamond, on='id', how='left')
test_res = test_res.reset_index(drop=True)
test_res.ec_blast = test_res.ec_blast.fillna('-')
test_res

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


,id,dmlf_isenzyme,dmlf_functions,sorted_ec,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast
0,WP_005054087.1,False,1,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9
1,WP_006079870.1,False,1,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0
2,WP_006079871.1,False,1,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0
3,WP_006080725.1,False,1,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0
4,WP_006080791.1,False,1,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0
...,...,...,...,...,...,...,...,...
4197,WP_238560620.1,False,1,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN
4198,WP_238560621.1,False,1,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN
4199,WP_238560622.1,False,1,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN
4200,WP_238560623.1,False,1,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN


### 7. Integrate Output

In [7]:
def ec_compare(ec_str1, ec_str2, sep=','):
    ec1= ec_str1.split(sep)
    ec2= ec_str2.split(sep)
    if set(ec1) == set(ec2):
        return True
    else:
        return False

test_res['ec_dmlf']=test_res.parallel_apply(lambda x: x.ec_blast if x.ec_dmlf=='-' else x.ec_dmlf, axis=1) #将序列比对到有EC号，DMLF预测成非酶的，进行EC填充
test_res=test_res[['id', 'ec_dmlf', 'ec_dmlf_topk', 'ec_blast','pident_blast']]
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0
...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN


## 8. DeepEC

In [8]:
# ! /home/shizhenkun/miniconda3/envs/deepec/bin/python ./baselines/deepec/deepec.py -i ./data/shewanella.faa -o /home/shizhenkun/codebase/DMLF/results/case_shewanella/deepec/

res_deepec= funclib.load_deepec_resluts(f'{cfg.RESULTSDIR}case_shewanella/deepec/DeepEC_Result.txt')
test_res= test_res.merge(res_deepec, on='id', how='left')
test_res.ec_deepec = test_res.ec_deepec.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-
...,...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-


# 9. ECPred

In [9]:
# ! java -jar /home/shizhenkun/codebase/DMLF/baselines/ECPred/ECPred.jar spmap /home/shizhenkun/codebase/DMLF/data/shewanella.faa /home/shizhenkun/codebase/DMLF/baselines/ECPred/ /home/shizhenkun/codebase/DMLF/results/case_gubang/ecpred/ /home/shizhenkun/codebase/DMLF/results/case_shewanella/ecpred/ecpred.tsv
res_ecpred =funclib.load_ecpred_res(f'{cfg.RESULTSDIR}ecpred/ecpred.txt')
test_res= test_res.merge(res_ecpred, on='id', how='left')
test_res.ec_ecpred = test_res.ec_ecpred.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN
...,...,...,...,...,...,...,...,...
4197,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN
4198,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN
4199,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN
4200,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN


## 10. CatFAM

In [10]:
# ! cd /home/shizhenkun/download/blast-2.2.26.bin/
# ! /home/shizhenkun/codebase/DMLF/baselines/catfam/source/catsearch.pl -d /home/shizhenkun/codebase/DMLF/baselines/catfam/CatFamDB/CatFam_v2.0/CatFam4D99R -i /home/shizhenkun/codebase/DMLF/data/shewanella.faa -o /home/shizhenkun/codebase/DMLF/results/case_shewanella/catfam/catfam.txt

res_catfam = funclib.load_catfam_res(f'{cfg.RESULTSDIR}/case_shewanella/catfam/catfam.txt')
test_res= test_res.merge(res_catfam, on='id', how='left')
test_res.ec_catfam = test_res.ec_catfam.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred,ec_catfam
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN,-
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN,-
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN,-
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN,-
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN,-
...,...,...,...,...,...,...,...,...,...
4227,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN,-
4228,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN,-
4229,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN,-
4230,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN,-


## 11. PRIAM

In [11]:
# ! java -Xmx128G -jar /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_search.jar -p /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_JAN18 -i /home/shizhenkun/codebase/DMLF/data/shewanella.faa -o /home/shizhenkun/codebase/DMLF/results/case_shewanella/priam --blast_path /home/shizhenkun/downloads/blast-2.2.13/bin -np 100
res_priam = funclib.load_praim_res(f'{cfg.RESULTSDIR}/case_shewanella/priam/PRIAM_20230128102132/ANNOTATION/sequenceECs.txt')
test_res= test_res.merge(res_priam, on='id', how='left')
test_res.ec_priam = test_res.ec_priam.fillna('-')
test_res

,id,ec_dmlf,ec_dmlf_topk,ec_blast,pident_blast,ec_deepec,ec_ecpred,pident_ecpred,ec_catfam,ec_priam
0,WP_005054087.1,-,"[(3.1.-.-, 0.1445898711681366), (3.1.22.-, 0.0...",-,40.9,-,-,NaN,-,-
1,WP_006079870.1,-,"[(3.1.26.5, 0.11312833428382874), (3.1.1.29, 0...",-,100.0,3.1.26.5,-,NaN,-,-
2,WP_006079871.1,-,"[(2.7.7.6, 0.6336387395858765), (3.1.26.5, 0.0...",-,100.0,-,-,NaN,-,-
3,WP_006080725.1,-,"[(3.1.26.5, 0.10884372144937515), (2.7.7.6, 0....",-,100.0,-,-,NaN,-,-
4,WP_006080791.1,-,"[(3.1.-.-, 0.9882077574729919), (4.2.99.18, 0....",-,100.0,-,-,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...
4227,WP_238560620.1,-,"[(3.1.-.-, 0.5166441202163696), (5.99.1.2, 0.0...",-,NaN,-,-,NaN,-,-
4228,WP_238560621.1,-,"[(3.1.-.-, 0.1078561320900917), (2.3.1.48, 0.0...",-,NaN,-,-,NaN,-,-
4229,WP_238560622.1,-,"[(3.1.-.-, 0.227324977517128), (3.1.21.-, 0.00...",-,NaN,-,-,NaN,-,-
4230,WP_238560623.1,-,"[(3.1.-.-, 0.9987987279891968), (3.4.24.-, 0.0...",-,NaN,-,-,NaN,-,-
